## 簡介

本課程將會涵蓋：
- 乜嘢係 function calling 同埋佢嘅應用場景
- 點樣用 OpenAI 去建立 function call
- 點樣將 function call 整合入你嘅應用程式

## 學習目標

完成呢課之後，你會識得同明白：

- 使用 function calling 嘅目的
- 用 OpenAI 服務設置 Function Call
- 為你嘅應用場景設計有效嘅 function calls


## 理解 Function Calls

今次課堂，我哋會為我哋嘅教育初創公司建立一個功能，等用戶可以用 chatbot 搵到技術課程。我哋會根據佢哋嘅技能水平、目前職位同埋有興趣嘅技術，推薦合適嘅課程。

為咗完成呢個功能，我哋會結合使用：
 - `OpenAI` 去為用戶創建一個聊天體驗
 - `Microsoft Learn Catalog API` 幫用戶根據佢哋嘅需求搵課程
 - `Function Calling` 將用戶嘅查詢傳送去一個 function，然後發出 API 請求

開始之前，先了解下點解我哋要用 function calling：

print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # 從 GPT 獲取新回應，佢可以睇到 function 嘅回應


print(second_response.choices[0].message)


### 點解要用 Function Calling

如果你已經完成咗呢個課程入面嘅其他課堂，你應該都明白用大型語言模型（LLMs）有幾強大。希望你都會留意到佢哋有啲限制。

Function Calling 係 OpenAI Service 嘅一個功能，專門用嚟解決以下幾個問題：

回應格式唔一致：
- 喺有 function calling 之前，大型語言模型嘅回應通常都冇結構同唔一致。開發者要寫好多複雜嘅驗證程式碼，去處理每一種唔同嘅輸出情況。

難以同外部數據整合：
- 喺有呢個功能之前，要將應用程式其他部分嘅數據帶入對話內容係好困難嘅。

透過標準化回應格式同方便整合外部數據，function calling 可以簡化開發流程，減少額外驗證邏輯嘅需要。

用戶以前問唔到類似「斯德哥爾摩而家天氣點呀？」咁嘅問題。因為模型只限於訓練時期嘅數據，冇辦法提供最新資訊。

我哋一齊睇下下面呢個例子，去說明呢個問題：

假設我哋想建立一個學生資料庫，方便推薦合適嘅課程畀佢哋。下面有兩個學生嘅描述，佢哋提供嘅資料都好相似。


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finshing his studies."

我哋想將呢個發送畀 LLM 去解析數據。之後可以用喺我哋嘅應用程式度，發送畀 API 或者儲存喺資料庫入面。

我哋一齊整兩個一模一樣嘅提示，去指示 LLM 我哋有興趣嘅資料係咩：


我們想將這個發送給一個大型語言模型，以解析對我們產品重要的部分。所以我們可以創建兩個相同的提示來指示大型語言模型：


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


在建立這兩個提示後，我們會用 `openai.ChatCompletion` 將它們發送到 LLM。我們將提示儲存在 `messages` 變數中，並將角色設為 `user`。這是為了模仿用戶向聊天機械人發送訊息。


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

現在我們可以同時向 LLM 發送兩個請求，並檢查收到的回應。


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

即使提示內容一樣，描述都差唔多，我哋都可以攞到唔同格式嘅 `Grades` 屬性。

如果你重複執行上面嗰個 cell，格式可以係 `3.7` 或者 `3.7 GPA`。

呢個係因為 LLM 係用文字提示嚟處理無結構嘅數據，回傳都係無結構嘅數據。我哋需要一個有結構嘅格式，咁先知道儲存或者用呢啲數據嘅時候會有咩預期。

用 functional calling 嘅話，我哋可以確保收到返嚟嘅數據係有結構嘅。用 function calling 嘅時候，LLM 其實唔會真係執行或者運行任何 function。相反，我哋會為 LLM 設計一個結構，等佢跟住嚟回應。之後我哋就可以根據呢啲有結構嘅回應，知道喺我哋嘅應用程式入面要執行邊個 function。


![函數調用流程圖](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.hk.png)


### 使用函數調用的應用場景

**調用外部工具**  
聊天機械人非常擅長回答用戶的問題。透過函數調用，聊天機械人可以根據用戶的訊息完成某些任務。例如，學生可以要求聊天機械人「發電郵給我的導師，說我需要更多協助」。這時可以調用 `send_email(to: string, body: string)` 這個函數。

**建立 API 或資料庫查詢**  
用戶可以用自然語言查找資料，系統會將其轉換成格式化的查詢或 API 請求。例如，老師可以問「哪些學生完成了上一份功課？」這可以調用名為 `get_completed(student_name: string, assignment: int, current_status: string)` 的函數。

**建立結構化數據**  
用戶可以將一段文字或 CSV 檔案，利用 LLM 抽取出重要資訊。例如，學生可以將一篇關於和平協議的維基百科文章轉換成 AI 閃卡。這可以透過 `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)` 這個函數來完成。


## 2. 建立你的第一個函數調用

建立函數調用的過程包括三個主要步驟：
1. 用你的函數清單同用戶訊息去調用 Chat Completions API
2. 閱讀模型回應，去執行動作，例如執行一個函數或者 API 調用
3. 用你函數回應的結果，再次調用 Chat Completions API，利用呢啲資料去生成回應俾用戶


![函數調用流程](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.hk.png)


### 函數調用的元素

#### 用戶輸入

第一步係建立一個用戶訊息。你可以動態咁攞一個文字輸入嘅值，或者直接喺呢度指定一個值。如果你第一次用 Chat Completions API，我哋需要定義訊息嘅 `role` 同埋 `content`。

`role` 可以係 `system`（用嚟設定規則）、`assistant`（即係模型）或者 `user`（即係最終用戶）。至於函數調用，我哋會將呢個設為 `user`，再加上一個示例問題。


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### 建立函數

接住嚟我哋會定義一個函數同埋佢嘅參數。呢度我哋只會用一個叫做 `search_courses` 嘅函數，但你都可以建立多個函數。

**重要**：函數會被包含喺系統訊息入面傳送畀 LLM，亦都會計算喺你可用嘅 token 數量之內。


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**定義**

函數定義結構有多個層級，每一層都有自己嘅屬性。以下係呢個巢狀結構嘅詳細說明：

**頂層函數屬性：**

`name` -  我哋想要調用嘅函數名稱。

`description` - 呢個係解釋函數點運作嘅描述。呢度要寫得具體同清晰。

`parameters` - 一個列出你想模型喺回應中產生嘅值同格式嘅清單。

**參數物件屬性：**

`type` -  參數物件嘅資料類型（通常係 "object"）

`properties` - 模型會用嚟生成回應嘅具體值清單

**個別參數屬性：**

`name` - 由屬性鍵隱含定義（例如 "role"、"product"、"level"）

`type` - 呢個特定參數嘅資料類型（例如 "string"、"number"、"boolean"）

`description` - 對呢個特定參數嘅描述

**可選屬性：**

`required` - 一個列出完成函數調用時必需參數嘅陣列


### 執行函數呼叫

定義好函數之後，我哋而家要喺 Chat Completion API 嘅請求入面加埋佢。只需要喺請求加多個 `functions`，即係 `functions=functions`。

另外，你都可以選擇將 `function_call` 設定為 `auto`。咁樣就會由 LLM 根據用戶訊息自動決定應該用邊個函數，而唔需要自己手動指定。


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

你可以見到，function 嘅名稱已經被呼叫，而由用戶訊息中，LLM 能夠搵到資料去填寫 function 嘅 arguments。


## 3. 將函數調用整合到應用程式內

當我們測試過由 LLM 格式化的回應之後，現在可以將它整合到應用程式內。

### 管理流程

要將這個功能整合到我們的應用程式，可以跟住以下步驟：

首先，呼叫 OpenAI 服務，並將回應訊息儲存在一個叫做 `response_message` 的變數內。


In [ ]:
response_message = response.choices[0].message

現在我們將定義一個函數，用來調用 Microsoft Learn API 以獲取課程列表：


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



作為最佳做法，我哋會先檢查模型有冇想調用某個函數。之後，我哋會建立一個可用嘅函數，並將佢配對到模型想調用嘅函數。

跟住，我哋會攞出函數嘅參數，並將佢哋對應到 LLM 傳過嚟嘅參數。

最後，我哋會加埋函數調用嘅訊息同埋 `search_courses` 訊息返回嘅數值。咁樣就可以畀 LLM 所有需要嘅資訊，用自然語言回應用戶。


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## 編碼挑戰

做得好！如果你想繼續學習 OpenAI Function Calling，可以試下建立呢個：https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst
 - 加入更多 function 參數，幫助學習者搵到更多合適課程。你可以喺呢度搵到可用嘅 API 參數：
 - 建立另一個 function call，收集學習者更多資料，例如佢哋嘅母語
 - 當 function call 或 API call 無法搵到合適課程時，加入錯誤處理



---

**免責聲明**：  
本文件經由 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 翻譯。我們致力於提供準確的翻譯，但請注意，自動翻譯可能會出現錯誤或不準確之處。原始語言的文件應被視為具權威性的來源。如涉及重要資訊，建議尋求專業人工翻譯。本翻譯所引致的任何誤解或曲解，我們概不負責。
